In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import glob
from keras.preprocessing import image
from scipy import spatial

**8.1.1 Prepare main image feature**

In [ ]:
# Load main image
main_image = cv2.imread('./pic.jpg')
main_image = cv2.cvtColor(main_image,cv2.COLOR_BGR2RGB)
h, w, c = main_image.shape

# Define tile size 
tile_width, tile_hight = 30,30
tile_size = (tile_width,tile_hight) # (Width,Height)

# Create main image feature grid of subimage
# Calculate average (mean) color of each subimage with tile_size

main_image_feature = cv2.resize(main_image,dsize = (w//tile_width,h//tile_hight))
for i in range(0,main_image_feature.shape[0]):
    for j in range(0,main_image_feature.shape[1]):
        main_image_feature[i,j] = main_image[i*tile_hight : (i+1)*tile_hight,j*tile_width : (j+1)*tile_width].mean(axis=(0,1))


In [ ]:
fig = plt.figure(figsize=(12,12))
ax = plt.subplot(1,2,1)
ax = plt.imshow(main_image)
ax = plt.title("Original")


ax = plt.subplot(1,2,2)
ax = plt.imshow(main_image_feature)
ax = plt.title("Main Image Features")

**8.1.2 Prepare tile images and features**

In [ ]:
filenames = glob.glob('Background/*.jpg', recursive=True)

# Load image files and Resize (h,w,ch) -> h = w < 100 , ch = 3 (R,G,B)
# Convert the image to an array
# Normalized color image
tiles = []
for i in filenames:
    img = image.load_img(i, target_size=tile_size)
    # img = image.img_to_array(img)
    # img = img / 255
    tiles.append(img)

# Convert to numpy array
# all_images = np.array(all_images)

# Create array of average(mean) color of each tile image

tile_features = []
for tile in tiles:
    mean_color = np.array(tile).mean(axis=(0,1))
    tile_features.append(mean_color)

**8.2 Image clustering and Matching using KNN with KDTree Algorithm**

In [ ]:
# Create KDTree of tile color features
tree = spatial.KDTree(tile_features)

#KNN (k=1) search for best mathched tiles with each subimage
for i in range(h):
    for j in range(w):
        closet = tree.query(main_image_feature[i,j])
        closest_tiles[i,j] = closest[1]

**8.3 Create image mosaic**

In [ ]:
# Fill each subimage with matched tile

# Offset of tile
x,y = i*tile_size[0], j*tile_size[1]

# Index of tile
index = closet_tiles[i,j]
main_photo[x:(x+tile_size[0]),y:(y+tile_size[1]),:] = tile[index]